In [1]:
import pandas as pd
import numpy as np

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [3]:
batch_size = 64             # Batch Size For Training
epochs = 100                # Number of Epcohes to Train for
laten_dim = 256             # Latent Dimensionality of the encoding space
num_samples = 10000         # Number of samples to train on

### Vectorize the data

In [5]:
input_texts = []
target_texts = []

input_char = set()
target_char = set()

with open(r"D:\Dig\New folder\fra.txt", encoding='utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines[: min(num_samples, len(lines)-1)]:
    input_text, target_text, _ = line.split('\t') # We use 'tab' as a target sequence character for target and
                                                  #          '\n' as 'end_sequence' character
    target_text = '\t' + target_text + '\n'                                                  
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_char:
            input_char.add(char)
            
    for char in target_text:
        if char not in target_char:
            target_char.add(char)

In [6]:
input_char  = sorted(list(input_char))
target_char = sorted(list(target_char))

num_encoder_tokens = len(input_char)
num_decoder_tokens = len(target_char)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
print("Number of Samples:", len(input_texts))

print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)

print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of Samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [9]:
input_token_index = dict([(char, i) for i, char in enumerate(input_char)])

target_token_index = dict([(char, i) for i, char in enumerate(target_char)])

In [10]:
input_token_index, target_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '5': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '%': 5,
  '&': 6,
  "'": 7,
  '(': 8,
  ')': 9,
  ',': 10,
  '-': 11,
  '.': 12,
  '0': 13,
  '1': 14,
  '2': 15,
  '3': 16,
  '5': 17,
  '8': 18,
  '9': 19,
  ':': 20,
  '?'

In [12]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [14]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i,t,target_token_index[char]] = 1.
        
        if t>0:
            # decoder_target_data will be ahead by one timestep and will not include the start char
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
    
encoder_input_data[0].shape

(15, 71)

### Define an input sequence and process it

In [15]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(laten_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# we discard 'encoder_outputs' and only keep the states

encoder_states = [state_h,state_c]

In [17]:
# Set up the decoder, using 'encoder_states' as initial state

decoder_inputs = Input(shape=(None,num_decoder_tokens))
# We setup our decoder to return full output sequence and to return internal states as well.
# We don't use the return states in the training model, but we will use them in inference

decoder_lstm = LSTM(laten_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 51s 411ms/step - loss: 1.1353 - accuracy: 0.7353 - val_loss: 1.1699 - val_accuracy: 0.7134
Epoch 2/100
125/125 [==============================] - 52s 414ms/step - loss: 0.8109 - accuracy: 0.7791 - val_loss: 0.8100 - val_accuracy: 0.7743
Epoch 3/100
125/125 [==============================] - 47s 377ms/step - loss: 0.6429 - accuracy: 0.8180 - val_loss: 0.6845 - val_accuracy: 0.8027
Epoch 4/100
125/125 [==============================] - 46s 369ms/step - loss: 0.5672 - accuracy: 0.8347 - val_loss: 0.6468 - val_accuracy: 0.8085
Epoch 5/100
125/125 [==============================] - 49s 389ms/step - loss: 0.5239 - accuracy: 0.8462 - val_loss: 0.5918 - val_accuracy: 0.8260
Epoch 6/100
125/125 [==============================] - 49s 395ms/step - loss: 0.4912 - accuracy: 0.8554 - val_loss: 0.5627 - val_accuracy: 0.8376
Epoch 7/100
125/125 [==============================] - 53s 422ms/step - loss: 0.4634 - accuracy: 0.8634 - val_loss: 0.5353 -

Epoch 57/100
125/125 [==============================] - 84s 673ms/step - loss: 0.0957 - accuracy: 0.9698 - val_loss: 0.5775 - val_accuracy: 0.8742
Epoch 58/100
125/125 [==============================] - 99s 790ms/step - loss: 0.0939 - accuracy: 0.9701 - val_loss: 0.5851 - val_accuracy: 0.8737
Epoch 59/100
125/125 [==============================] - 83s 665ms/step - loss: 0.0913 - accuracy: 0.9707 - val_loss: 0.5909 - val_accuracy: 0.8740
Epoch 60/100
125/125 [==============================] - 73s 584ms/step - loss: 0.0894 - accuracy: 0.9715 - val_loss: 0.5954 - val_accuracy: 0.8730
Epoch 61/100
125/125 [==============================] - 86s 688ms/step - loss: 0.0879 - accuracy: 0.9720 - val_loss: 0.5970 - val_accuracy: 0.8732
Epoch 62/100
125/125 [==============================] - 78s 624ms/step - loss: 0.0859 - accuracy: 0.9726 - val_loss: 0.6028 - val_accuracy: 0.8728
Epoch 63/100
125/125 [==============================] - 79s 636ms/step - loss: 0.0841 - accuracy: 0.9731 - val_loss: 0

In [23]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_states_input_h = Input(shape=(laten_dim,))
decoder_states_input_c = Input(shape=(laten_dim,))

decoder_states_input = [decoder_states_input_h, decoder_states_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_input)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs]+decoder_states_input, [decoder_outputs]+decoder_states)


reverse_input_char_index = dict((i,char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i,char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decode_sentence = decode_sequence(input_seq)
    print('-')
    
    print("input state:", input_text[seq_index])
    print("decoded sentence:", decode_sentence)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_1:0", shape=(None, None, 71), dtype=float32) at layer "input_1". The following previous layers were accessed without issue: []